In [1]:
import spacy
import re
import en_core_web_sm
import math
import networkx as nx
import numpy as np
nlp = spacy.load("en_core_web_sm")

In [2]:
##Open text with the first book
##Chapters is a list which contains all the chapters ; we the .split() function which detect all times with 5 new line

with open('GOT1.txt','r') as fp:
    book = fp.read()
    chapters = book.split('\n\n\n\n\n')
    sm = []
    for i in range(0,len(chapters)):
        name = [word for word in chapters[i].split('\n') if len(word.strip())>0][0]
        print (i+1,name.title())
        chapter = ' '.join([word.replace('\t','').replace('"','') for word in chapters[i].split('\n') if len(word.strip())>0][1:])
                           
        doc1 = nlp(chapter)
        for ent in doc1.ents:
            if ent.label_ == 'PERSON' :
                sm.append((ent.text, ent.start_char, ent.end_char))

1 Prologue
2 Bran
3 Catelyn
4 Daenerys
5 Eddard
6 Jon
7 Catelyn
8 Arya
9 Bran
10 Tyrion
11 Jon
12 Daenerys
13 Eddard
14 Tyrion
15 Catelyn
16 Sansa
17 Eddard
18 Bran
19 Catelyn
20 Jon
21 Eddard
22 Tyrion
23 Arya
24 Daenerys
25 Bran
26 Eddard
27 Jon
28 Eddard
29 Catelyn
30 Sansa
31 Eddard
32 Tyrion
33 Arya
34 Eddard
35 Catelyn
36 Eddard
37 Daenerys
38 Bran
39 Tyrion
40 Eddard
41 Catelyn
42 Jon
43 Tyrion
44 Eddard
45 Sansa
46 Eddard
47 Daenerys
48 Eddard
49 Jon
50 Eddard
51 Arya
52 Sansa
53 Jon
54 Bran
55 Daenerys
56 Catelyn
57 Tyrion
58 Sansa
59 Eddard
60 Catelyn
61 Jon
62 Daenerys
63 Tyrion
64 Catelyn
65 Daenerys
66 Arya
67 Bran
68 Sansa
69 Daenerys
70 Tyrion
71 Jon
72 Catelyn
73 Daenerys


In [3]:
import pickle
with open ('GOT-characters-raw.pickle','rb') as ch:
    characters=pickle.load(ch)  
len(characters.keys())

340

In [4]:
entities = set()
for chapter in characters.values():
    for entity in chapter:
        name=entity[0]
        entities.add(name)
print(len(entities))

5522


In [5]:
GOT=nx.Graph()



In [6]:
print(nx.info(GOT))

Name: 
Type: Graph
Number of nodes: 0
Number of edges: 0



In [7]:
chapter=characters['1–1-Bran']
chapter.sort(key=lambda a:a[1],reverse=False)

def get_edges_from_chapter(chapter):
    chapter.sort(key = lambda a:a[1], reverse = False) #trier perso par postiion dans le texte

    potential_edges = []

    for i in range(0, len(chapter)): #balaye tout les perso
        name1, start1, end1 = chapter[i] #rebalaye et lien avec ceux d'après
        for j in range(i+1, len(chapter)):
            name2, start2, end2 = chapter[j]
            if name2 != name1:
                potential_edges.append((name1,name2,{'dist':start2-end1}))
    edges = {}
    for name1,name2,value in potential_edges:
        if (name1,name2) not in edges.keys():
            if (name2,name1) not in edges.keys():
                edges[(name1,name2)] = {'values':[value['dist']]}
            else:
                edges[(name2,name1)]['values'].append(value['dist'])
        else:
            edges[(name1,name2)]['values'].append(value['dist'])

    for k,v in edges.items():
        #edges[k]['weight']=np.percentile(v['values'],10)
        edges[k]['weight']=len([a for a in v['values'] if a < 1000])
        #print (k, edges[k]['weight'])
    
    ready_for_nx =[]
    for k,v in edges.items():
        name1 = k[0]
        name2 = k[1]
        weight = v['weight']
        if weight>0:
            ready_for_nx.append((name1, name2, weight))
    return ready_for_nx

In [8]:
for chapter in characters.values():
    GOT.add_edges_from(get_edges_from_chapter(chapter))

TypeError: 'int' object is not iterable

In [38]:
print(nx.info(GOT))

Name: 
Type: Graph
Number of nodes: 2
Number of edges: 0
Average degree:   0.0000
